In [1370]:
import warnings
import pandas as pd
import numpy as np

import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import PowerTransformer

In [1371]:
train_data = pd.read_csv('./mod_04_hw_train_data.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           249 non-null    object 
 1   Phone_Number   249 non-null    object 
 2   Experience     247 non-null    float64
 3   Qualification  248 non-null    object 
 4   University     249 non-null    object 
 5   Role           246 non-null    object 
 6   Cert           247 non-null    object 
 7   Date_Of_Birth  249 non-null    object 
 8   Salary         249 non-null    int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 17.6+ KB


In [1372]:
test_data = pd.read_csv('./mod_04_hw_valid_data.csv')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           7 non-null      object
 1   Phone_Number   7 non-null      object
 2   Experience     7 non-null      int64 
 3   Qualification  7 non-null      object
 4   University     7 non-null      object
 5   Role           7 non-null      object
 6   Cert           7 non-null      object
 7   Date_Of_Birth  7 non-null      object
 8   Salary         7 non-null      int64 
dtypes: int64(2), object(7)
memory usage: 636.0+ bytes


In [1373]:
train_data.nunique()

Name             248
Phone_Number     249
Experience         5
Qualification      3
University         3
Role               3
Cert               2
Date_Of_Birth    247
Salary            84
dtype: int64

In [1374]:
train_data.drop(['Name', 'Phone_Number', 'Date_Of_Birth'], axis=1, inplace=True)
test_data.drop(['Name', 'Phone_Number', 'Date_Of_Birth'], axis=1, inplace=True)

In [1375]:
y_train = train_data['Salary']
y_test = test_data['Salary']
X_train = train_data.drop('Salary', axis=1)
X_test = test_data.drop('Salary', axis=1)


In [1376]:
X_train = X_train.dropna()
X_test = X_test.dropna()

y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

In [1377]:
X_train.select_dtypes(include='object').nunique()

Qualification    3
University       3
Role             3
Cert             2
dtype: int64

In [1378]:
categorical_features = X_train.select_dtypes(include=['object']).columns

In [1379]:
target_encoder = ce.TargetEncoder(cols=categorical_features)

X_train[categorical_features] = target_encoder.fit_transform(X_train[categorical_features], y_train)

X_test[categorical_features] = target_encoder.transform(X_test[categorical_features])

In [1380]:
transformer = PowerTransformer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

In [1381]:
model = KNeighborsRegressor(n_neighbors=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE: {mape * 100:.2f}%')

MAPE: 4.56%


### Висновок:
1. Якість моделі:

    - Поточне значення MAPE = 4.56% вказує на те, що середня абсолютна похибка моделі становить 4.56% від реальних значень. Це є досить гарним результатом для задачі регресії, оскільки прогнозування заробітної плати з такою точністю є прийнятним для більшості прикладних задач.
2. Основні аспекти аналізу моделі:

    - n_neighbors = 10: Вибір 10 сусідів є хорошим компромісом між точністю та узагальненням. 10 дав найкращий результат в діапазоні до 21.
    - Метрика відстані: Використовується евклідова відстань за замовчуванням. Інші варанти не дали покращення моделі.
    - Попередня обробка даних: Використання PowerTransformer покращило якість, адже ця трансформація допомагає зробити дані більш нормалізованими.